# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [34]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2409560068686,
 'week52high': 142.84,
 'week52low': 58.22,
 'week52change': 0.7629746983966401,
 'sharesOutstanding': 17040198137,
 'float': 0,
 'avg10Volume': 107957427,
 'avg30Volume': 113192011,
 'day200MovingAvg': 120.67,
 'day50MovingAvg': 132.56,
 'employees': 0,
 'ttmEPS': 3.29,
 'ttmDividendRate': 0.827940529443904,
 'dividendYield': 0.00605976716780306,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-25',
 'nextEarningsDate': '0',
 'peRatio': 41.60940845872534,
 'beta': 1.2065964073930853,
 'maxChangePercent': 52.881558995251986,
 'year5ChangePercent': 5.06882724636891,
 'year2ChangePercent': 2.752208526447296,
 'year1ChangePercent': 0.7894167986506816,
 'ytdChangePercent': 0.04859335252412876,
 'month6ChangePercent': 0.4556649193199966,
 'month3ChangePercent': 0.2114833361553604,
 'month1ChangePercent': 0.05454159905687447,
 'day30ChangePercent': 0.0635411872922723,
 'day5ChangePercent': 0.09430746361317491}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['ytdChangePercent']

0.04859335252412876

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings[:1]:
 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['close'],
                data[symbol]['quote']['ytdChange'],
                'N/A'
            ],
            index = my_columns),
                ignore_index = True
            )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,131.23,0.065040,N/A
1,AAL,15.87,0.002618,N/A
2,AAP,164.5,0.022226,N/A
3,AAPL,140.93,0.064933,N/A
4,ABBV,113.42,0.043553,N/A
...,...,...,...,...
95,CINF,90.86,-0.011767,N/A
96,CL,81.13,-0.091805,N/A
97,CLX,208.87,0.028751,N/A
98,CMA,63.09,0.116164,N/A


## Removing Low-Momentum Stocks
The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,29,AMAT,111.53,0.224357,N/A
1,23,ALB,179.23,0.167560,N/A
2,58,BBY,118.81,0.160581,N/A
3,16,AES,28,0.151517,N/A
4,42,APA,17.19,0.143168,N/A
5,5,ABC,109.76,0.131729,N/A
6,89,CF,44.9,0.120936,N/A
7,98,CMA,63.09,0.116164,N/A
8,33,AMGN,259.2,0.111139,N/A
9,62,BIIB,275.31,0.100582,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter size of your portfolio')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number')
        print('Please try again:')
        portfolio_size = input('Enter the size of your portfolio')
        
portfolio_input()
print(portfolio_size)

Enter size of your portfolio100000000
100000000


In [10]:
portfolio_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(portfolio_size/final_dataframe.loc[i, 'Price']
)    
final_dataframe

/Users/Haruna/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,29,AMAT,111.53,0.224357,17932
1,23,ALB,179.23,0.167560,11158
2,58,BBY,118.81,0.160581,16833
3,16,AES,28,0.151517,71428
4,42,APA,17.19,0.143168,116346
5,5,ABC,109.76,0.131729,18221
6,89,CF,44.9,0.120936,44543
7,98,CMA,63.09,0.116164,31700
8,33,AMGN,259.2,0.111139,7716
9,62,BIIB,275.31,0.100582,7264


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [31]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Year Price Return',
    'One-Year Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings[:1]:
    for symbol in symbol_string.split(','):
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        hqm_dataframe = hqm_dataframe.append(
            pd.Series([
            symbol,
            data[symbol]['quote']['close'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Year Price Return,One-Year Return Percentile
0,A,130.95,N/A,0.429548,N/A,0.327648,N/A,0.191463,N/A,0.429548,N/A
1,AAL,15.84,N/A,-0.424608,N/A,0.409276,N/A,0.204744,N/A,-0.424608,N/A
2,AAP,166.72,N/A,0.121062,N/A,0.102709,N/A,0.074234,N/A,0.121062,N/A
3,AAPL,140.76,N/A,0.788131,N/A,0.439549,N/A,0.206529,N/A,0.788131,N/A
4,ABBV,113.79,N/A,0.343014,N/A,0.171419,N/A,0.344851,N/A,0.343014,N/A
...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,88.2,N/A,-0.159578,N/A,0.110387,N/A,0.146558,N/A,-0.159578,N/A
96,CL,81.65,N/A,0.140795,N/A,0.068783,N/A,-0.008429,N/A,0.140795,N/A
97,CLX,202.64,N/A,0.310823,N/A,-0.103003,N/A,-0.031129,N/A,0.310823,N/A
98,CMA,64.15,N/A,0.000575,N/A,0.780159,N/A,0.410830,N/A,0.000575,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [50]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,One-Yaer Return Percentile
0,A,130.95,N/A,0.429548,85,0.327648,76,0.191463,68,0.429548,85,0.0
1,AAL,15.84,N/A,-0.424608,3,0.409276,83,0.204744,70,-0.424608,3,0.0
2,AAP,166.72,N/A,0.121062,53,0.102709,31,0.074234,35,0.121062,53,0.0
3,AAPL,140.76,N/A,0.788131,94,0.439549,86,0.206529,71,0.788131,94,0.0
4,ABBV,113.79,N/A,0.343014,78,0.171419,52,0.344851,87,0.343014,78,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,88.2,N/A,-0.159578,14,0.110387,32,0.146558,56,-0.159578,14,0.0
96,CL,81.65,N/A,0.140795,54,0.068783,28,-0.008429,14,0.140795,54,0.0
97,CLX,202.64,N/A,0.310823,74,-0.103003,4,-0.031129,7,0.310823,74,0.0
98,CMA,64.15,N/A,0.000575,29,0.780159,99,0.410830,92,0.000575,29,0.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [52]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,One-Yaer Return Percentile,HQM Score
0,A,130.95,N/A,0.429548,85,0.327648,76,0.191463,68,0.429548,85,0.0,78.50
1,AAL,15.84,N/A,-0.424608,3,0.409276,83,0.204744,70,-0.424608,3,0.0,39.75
2,AAP,166.72,N/A,0.121062,53,0.102709,31,0.074234,35,0.121062,53,0.0,43.00
3,AAPL,140.76,N/A,0.788131,94,0.439549,86,0.206529,71,0.788131,94,0.0,86.25
4,ABBV,113.79,N/A,0.343014,78,0.171419,52,0.344851,87,0.343014,78,0.0,73.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,88.2,N/A,-0.159578,14,0.110387,32,0.146558,56,-0.159578,14,0.0,29.00
96,CL,81.65,N/A,0.140795,54,0.068783,28,-0.008429,14,0.140795,54,0.0,37.50
97,CLX,202.64,N/A,0.310823,74,-0.103003,4,-0.031129,7,0.310823,74,0.0,39.75
98,CMA,64.15,N/A,0.000575,29,0.780159,99,0.410830,92,0.000575,29,0.0,62.25


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [60]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index()
hqm_dataframe

/Users/Haruna/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Ticker,Price,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,One-Yaer Return Percentile,HQM Score
23,ALB,175.24,114129,1.182635,99,0.994399,100,0.847950,100,1.182635,99,0.0,99.50
29,AMAT,108.35,184587,0.702159,92,0.667474,95,0.747578,98,0.702159,92,0.0,94.25
45,APTV,144.15,138744,0.550362,88,0.693801,96,0.404126,91,0.550362,88,0.0,90.75
24,ALGN,534.66,37407,0.957243,98,0.699301,97,0.187258,67,0.957243,98,0.0,90.00
78,CARR,41.47,482276,2.378200,100,0.503591,89,0.175671,64,2.378200,100,0.0,88.25
50,AVGO,488.09,40976,0.561334,89,0.512430,91,0.264033,83,0.561334,89,0.0,88.00
85,CDNS,139.5,143369,0.847430,96,0.339824,78,0.263045,81,0.847430,96,0.0,87.75
28,ALXN,163.87,122048,0.441354,86,0.520964,93,0.340886,86,0.441354,86,0.0,87.75
16,AES,27.3,732601,0.377821,80,0.774450,98,0.344959,88,0.377821,80,0.0,86.50
3,AAPL,140.76,142086,0.788131,94,0.439549,86,0.206529,71,0.788131,94,0.0,86.25


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [56]:
portfolio_input()

Enter size of your portfolio1000000000


In [59]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = position_size / hqm_dataframe.loc[i, 'Price']

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Month Price Return,One-Month Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Year Price Return,One-Year Return Percentile,One-Yaer Return Percentile,HQM Score
23,ALB,175.24,114129,1.182635,99,0.994399,100,0.847950,100,1.182635,99,0.0,99.50
29,AMAT,108.35,184587,0.702159,92,0.667474,95,0.747578,98,0.702159,92,0.0,94.25
45,APTV,144.15,138744,0.550362,88,0.693801,96,0.404126,91,0.550362,88,0.0,90.75
24,ALGN,534.66,37407,0.957243,98,0.699301,97,0.187258,67,0.957243,98,0.0,90.00
78,CARR,41.47,482276,2.378200,100,0.503591,89,0.175671,64,2.378200,100,0.0,88.25
50,AVGO,488.09,40976,0.561334,89,0.512430,91,0.264033,83,0.561334,89,0.0,88.00
85,CDNS,139.5,143369,0.847430,96,0.339824,78,0.263045,81,0.847430,96,0.0,87.75
28,ALXN,163.87,122048,0.441354,86,0.520964,93,0.340886,86,0.441354,86,0.0,87.75
16,AES,27.3,732601,0.377821,80,0.774450,98,0.344959,88,0.377821,80,0.0,86.50
3,AAPL,140.76,142086,0.788131,94,0.439549,86,0.206529,71,0.788131,94,0.0,86.25


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [95]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [96]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [97]:
column_formats ={ 
    'A': ['Ticker', string_template], 
    'B': ['Price', dollar_template], 
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['One-Month Price Return', percent_template], 
    'E': ['One-Month Return Percentile', percent_template], 
    'F': ['Six-Month Price Return', percent_template], 
    'G': ['Six-Month Return Percentile', percent_template], 
    'H': ['Three-Month Price Return', percent_template], 
    'I': ['Three-Month Return Percentile', percent_template], 
    'J': ['One-Year Price Return', percent_template], 
    'K': ['One-Year Return Percentile', percent_template], 
    'L': ['HQM Score', percent_template] 
}

for column in column_formats.keys(): 
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1]) 
    writer.sheets['Momentum Strategy'].write(f'{column}1', 25, column_formats[column][1]) 

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [98]:
writer.save()